# Configuração, Inicialização e Carregamento de Dados

In [1]:
!pip install pyspark # Instala a biblioteca PySpark no ambiente de execução

In [2]:
import pyspark # Importa a biblioteca principal do PySpark
from pyspark.sql import SparkSession # Importa a classe SparkSession, usada para iniciar a sessão Spark.
from pyspark.sql.functions import col, min, max # col (referência a coluna), e as funções de agregação min e max, usadas para a normalização manual
from pyspark.ml.feature import StringIndexer, MinMaxScaler, PCA, VectorAssembler # Importa as classes essenciais para pré-processamento de Machine Learning: StringIndexer (codificação categórica), MinMaxScaler (normalização), PCA (redução de dimensionalidade) e VectorAssembler (agrupamento de features)

In [3]:
from google.colab import drive # Importa o módulo drive do Google Colab.
drive.mount('/content/drive') # Monta o Google Drive no ambiente.

Mounted at /content/drive


In [4]:
spark = SparkSession.builder.getOrCreate() # Cria ou obtém a instância da SparkSession

In [5]:
produtos = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/produtos.csv", header=True, inferSchema=True) # Lê o DataFrame produtos
pagamentos_pedido = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/pagamentos_pedido.csv", header=True, inferSchema=True) # Lê o DataFrame pagamentos_pedido

# Codificação de Variáveis Categóricas (StringIndexer)

In [6]:
# valores categóricos
indexador = StringIndexer(inputCol = 'categoria_produto', outputCol='categoria_produto_index') # Define que a coluna de entrada (inputCol) é 'categoria_produto' e a coluna de saída (outputCol) será 'categoria_produto_index'
modelo_indexer = indexador. fit(produtos) # O fit calcula os mapeamentos de strings para índices com base nos dados do produtos
produtos_indexado_df = modelo_indexer.transform(produtos) # Aplica o mapeamento aprendido para criar a nova coluna de índices no DataFrame

produtos_indexado_df.show() # Exibe o resultado da codificação, mostrando a nova coluna numérica

+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+-----------------------+
|          id_produto|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|categoria_produto_index|
+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+-----------------------+
|1e9e8ef04dbcff454...|          perfumaria|                  40|                      287|                       1|           225|                    16|               10|                14|                   11.0|
|3aa071139cb16b67c...|               artes|                  44|                      276|                       1|          1000|          

# Normalização de Features (Min-Max Scaling)

In [7]:
# Normalização manual
min_val, max_val = pagamentos_pedido.select(min('valor_pagamento'), max('valor_pagamento')).first() # Obtém os valores mínimo e máximo da coluna 'valor_pagamento'. O .first() retorna o resultado como uma tupla
pagamentos_normalizado_df = pagamentos_pedido.withColumn('valor_pagamento_normalizado', (col('valor_pagamento') - min_val) / (max_val - min_val)) # Cria a coluna 'valor_pagamento_normalizado' aplicando manualmente a fórmula Min-Max Scaling: $(X - \text{Min}) / (\text{Max} - \text{Min})$

min_parc, max_parc = pagamentos_normalizado_df.select(min('parcelas_pagamento'), max('parcelas_pagamento')).first() # Obtém os valores mínimo e máximo da coluna 'parcelas_pagamento'
pagamentos_normalizado_df = pagamentos_normalizado_df.withColumn('parcelas_pagamento_normalizado', (col('parcelas_pagamento') - min_parc) / (max_parc - min_parc)) # Cria a coluna 'parcelas_pagamento_normalizado' aplicando a mesma fórmula de Min-Max Scaling

pagamentos_normalizado_df.show() # Exibe os dados de pagamento com as novas colunas normalizadas

+--------------------+-------------------+--------------+------------------+---------------+---------------------------+------------------------------+
|           id_pedido|sequencia_pagamento|tipo_pagamento|parcelas_pagamento|valor_pagamento|valor_pagamento_normalizado|parcelas_pagamento_normalizado|
+--------------------+-------------------+--------------+------------------+---------------+---------------------------+------------------------------+
|b81ef226f3fe1789b...|                  1|   credit_card|                 8|          99.33|       0.007269424652080491|            0.3333333333333333|
|a9810da82917af2d9...|                  1|   credit_card|                 1|          24.39|       0.001784971984941...|          0.041666666666666664|
|25e8ea4e93396b6fa...|                  1|   credit_card|                 1|          65.71|       0.004808958963940492|          0.041666666666666664|
|ba78997921bbcdc13...|                  1|   credit_card|                 8|         107

In [8]:
# junta as colunas numéricas de dimensão e peso (peso_produto_g, comprimento_produto_cm, etc.) em uma única coluna vetorial chamada características, que é o formato exigido pelos transformadores do MLlib
montar_vetor = VectorAssembler(
    inputCols=["peso_produto_g","comprimento_produto_cm","altura_produto_cm","largura_produto_cm"], #
    outputCol="características",
    handleInvalid="skip"
)

produtos_caracteristicas_df = montar_vetor.transform(produtos) # Aplica o VectorAssembler, criando o vetor de features
produtos_caracteristicas_df.show()

+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+--------------------+
|          id_produto|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|     características|
+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+--------------------+
|1e9e8ef04dbcff454...|          perfumaria|                  40|                      287|                       1|           225|                    16|               10|                14|[225.0,16.0,10.0,...|
|3aa071139cb16b67c...|               artes|                  44|                      276|                       1|          1000|                    30

In [9]:
from pyspark.ml.feature import VectorAssembler, MinMaxScaler

# normalizar
scaler = MinMaxScaler(inputCol='características', outputCol='caracteristicas_normalizado') # Define o transformador que aplicará a normalização Min-Max no vetor de features.
modelo_scaler = scaler.fit(produtos_caracteristicas_df) # O fit calcula os valores mínimos e máximos dentro do vetor de features

produtos_caracteristicas_normalizado_df = modelo_scaler.transform(produtos_caracteristicas_df) # Aplica a normalização, criando um novo vetor chamado caracteristicas_normalizado
produtos_caracteristicas_normalizado_df.show(truncate=False) # Exibe o resultado da normalização automática.

+--------------------------------+--------------------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+------------------------+-----------------------------------------------------------------------------------+
|id_produto                      |categoria_produto               |tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|características         |caracteristicas_normalizado                                                        |
+--------------------------------+--------------------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+------------------------+-----------------------------------------------------------------------------------+
|1e9e8ef04dbcff4541ed26657ea

# Redução de Dimensionalidade (PCA)

In [10]:
#reduzir dimensionalidade
pca = PCA(k=2, inputCol='caracteristicas_normalizado', outputCol='caracteristicas_pca') # Define o PCA para reduzir o vetor de features para 2 componentes principais (k=2). A entrada é o vetor normalizado
modelo_pca = pca.fit(produtos_caracteristicas_normalizado_df) # O fit calcula os vetores e valores próprios (eigenvectors e eigenvalues) dos dados
produtos_caracteristicas_reduzida_df = modelo_pca.transform(produtos_caracteristicas_normalizado_df) # Aplica a transformação, projetando o vetor original em um novo vetor de 2 dimensões (caracteristicas_pca)

produtos_caracteristicas_reduzida_df.show(truncate=False) # Exibe o resultado final com a dimensionalidade reduzida

+--------------------------------+--------------------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+------------------------+-----------------------------------------------------------------------------------+--------------------------------------------+
|id_produto                      |categoria_produto               |tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|características         |caracteristicas_normalizado                                                        |caracteristicas_pca                         |
+--------------------------------+--------------------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+------------------------+-----------------------

# Exportação e Encerramento

In [11]:
produtos_caracteristicas_reduzida_df.write.mode('overwrite').option('header','true').parquet('/content/drive/MyDrive/Material de apoio - M27/output/produtos_preprocessados_parquet') # Salva o DataFrame de produtos (com as características PCA) no formato Parquet no Google Drive

df = spark.read.option('header','true').parquet('/content/drive/MyDrive/Material de apoio - M27/output/produtos_preprocessados_parquet') # Lê o arquivo Parquet de volta
df.show(5) # Exibe as 5 primeiras linhas
df.printSchema() # o schema do DataFrame salvo

+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+--------------------+---------------------------+--------------------+
|          id_produto|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|     características|caracteristicas_normalizado| caracteristicas_pca|
+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+--------------------+---------------------------+--------------------+
|1e9e8ef04dbcff454...|          perfumaria|                  40|                      287|                       1|           225|                    16|               10|                14|[225.0,16.0,10.0,...|      

In [12]:
pagamentos_normalizado_df.write.mode('overwrite').option('header','true').parquet("/content/drive/MyDrive/Material de apoio - M27/output/pagamento_pedido_preprocessados_parquet") # Salva o DataFrame de pagamentos (com normalização manual) no formato Parquet

spark.read.option('header','true').parquet("/content/drive/MyDrive/Material de apoio - M27/output/pagamento_pedido_preprocessados_parquet") # Lê o arquivo Parquet de pagamentos salvo (sem mostrar o resultado).

DataFrame[id_pedido: string, sequencia_pagamento: int, tipo_pagamento: string, parcelas_pagamento: int, valor_pagamento: double, valor_pagamento_normalizado: double, parcelas_pagamento_normalizado: double]

In [13]:
spark.stop() # Encerra a SparkSession e libera os recursos